In [27]:
import nuclio

In [28]:
# nuclio: start-code

In [29]:
import time
from os import path, getenv
import os
from mlrun import load_project

In [30]:
def init_context(context):
    setattr(context.user_data,'project',load_project('/User/projects/deploy',url='/User/igztraining/mlrun/Day-4/conf/project.yaml'))
    pass

In [31]:
def handler(context,event):
    if 'source_file' in event.fields:
        source_file = event.fields['source_file']
    else:
        source_file = 'demo.csv'
    os.chdir('/User/igztraining/mlrun/Day-4/conf/')
    arguments = {"source_file" : "/User/igztraining/mlrun/Day-4/data/%s"% source_file}
    run_id = context.user_data.project.run(
    'main',
    arguments=arguments, 
    artifact_path=path.join('/User/pipeline','{{workflow.uid}}'), 
    dirty=True)
    
    return


In [32]:
# nuclio: end-code

In [33]:
from mlrun import code_to_function, mount_v3io

In [34]:
fn = code_to_function(kind='nuclio')

In [35]:
fn.apply(mount_v3io())
fn.spec.build.commands = ['pip install mlrun==0.5.1']

In [36]:
trigger_spec={
              'kind': 'cron',
            "maxWorkers" : 1,
            'attributes': {"interval": '10m'
              }
            }
fn.add_trigger('input-stream',trigger_spec)


In [37]:
fn.spec.min_replicas = 1
fn.spec.max_replicas = 1

In [38]:
addr = fn.deploy(project='ecolab-nuclio')

> 2020-09-30 12:07:13,666 [info] deploy started
[nuclio] 2020-09-30 12:07:14,764 (info) Build complete
[nuclio] 2020-09-30 12:07:24,876 (info) Function deploy complete
[nuclio] 2020-09-30 12:07:24,884 done updating ecolab-nuclio-triggerpipeline, function address: 3.129.97.78:30997
